In [ ]:
# import findspark
# findspark.init()

# import pyspark
# from pyspark.sql import SparkSession

In [ ]:
# spark = SparkSession.builder \
# .master('local') \
# .appName('read_sensor_data') \
# .config('spark.jars.packages", "com.crealytics:spark-excel_2.11:0.12.2') \
# .getOrCreate()

In [1]:
import pandas as pd

In [2]:
sensor1_file = 'data_engineer_test/sensor_1.xlsx'
sensor2_file = 'data_engineer_test/sensor_2.csv'
sensor4_file = 'data_engineer_test/sensor_4.parquet'
sensor4_dd_file = 'data_engineer_test/sensor_4_diff_date.pickle'
sensor5_file = 'data_engineer_test/sensor_5.json'

In [3]:
df = pd.DataFrame()

In [4]:
# sensor 1 - xlsx
s1_pd = pd.read_excel(sensor1_file)
print(s1_pd.shape)

# # s1 pyspark
# s1_sp = spark.createDataFrame(s1_pd)

(6530, 4)


In [5]:
# sensor 2 - csv
s2_pd = pd.read_csv(sensor2_file)
print(s2_pd.shape)

(7076, 4)


In [6]:
# sensor 4 - parquet
s4_pd = pd.read_parquet(sensor4_file)
print(s4_pd.shape)

(715, 4)


In [7]:
# sensor 4 diff date - pickle
s4_dd_pd = pd.read_pickle(sensor4_dd_file)
print(s4_dd_pd.shape)

(4551, 4)


In [8]:
# sensor 5 - json
s5_pd = pd.read_json(sensor5_file, orient='index')
print(s5_pd.shape)

(7185, 4)


In [9]:
df = pd.concat([s1_pd, s2_pd, s4_pd, s4_dd_pd, s5_pd])
print(df.shape)

(26057, 4)


In [10]:
df_good = df[df['tag_quality']=='Good']
df_good['created_timestamp'] = pd.to_datetime(df_good['created_timestamp'])
df_good

/var/folders/pd/kpj3vxxs34nf_nxz3l8kbb6r0000gn/T/ipykernel_8625/1597808014.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_good['created_timestamp'] = pd.to_datetime(df_good['created_timestamp'])


,created_timestamp,tag_key,tag_val,tag_quality
0,2021-07-05 00:00:24.987000+00:00,sens_1,31.0,Good
1,2021-07-05 00:00:27.833000+00:00,sens_1,29.0,Good
2,2021-07-05 00:01:17.633000+00:00,sens_1,31.0,Good
3,2021-07-05 00:01:25.107000+00:00,sens_1,29.0,Good
5,2021-07-05 00:04:44.929000+00:00,sens_1,29.0,Good
...,...,...,...,...
7180,2021-06-08 23:59:23.932000+00:00,sens_5,32.0,Good
7181,2021-06-08 23:59:31.043000+00:00,sens_5,33.0,Good
7182,2021-06-08 23:59:33.890000+00:00,sens_5,32.0,Good
7183,2021-06-08 23:59:44.273000+00:00,sens_5,33.0,Good


In [38]:
df_pivoted = pd.pivot_table(df_good, values='tag_val', index='created_timestamp', columns='tag_key')
df_pivoted = df_pivoted.sort_index(axis=0)
df_pivoted

tag_key,sens_1,sens_2,sens_4,sens_5
created_timestamp,,,,
2021-06-08 00:00:20.201000+00:00,NaN,NaN,NaN,45.0
2021-06-08 00:00:24.469000+00:00,NaN,NaN,NaN,46.0
2021-06-08 00:01:05.046000+00:00,NaN,NaN,NaN,45.0
2021-06-08 00:01:19.484000+00:00,NaN,NaN,NaN,46.0
2021-06-08 00:01:26.833000+00:00,NaN,NaN,NaN,45.0
...,...,...,...,...
2021-08-15 02:23:47.042000+00:00,NaN,NaN,0.0,NaN
2021-08-15 02:23:49.790000+00:00,NaN,NaN,1.0,NaN
2021-08-15 02:23:53.729000+00:00,NaN,NaN,0.0,NaN


In [39]:
df_ffill = df_pivoted.fillna(method='ffill')
df_ffill

tag_key,sens_1,sens_2,sens_4,sens_5
created_timestamp,,,,
2021-06-08 00:00:20.201000+00:00,NaN,NaN,NaN,45.0
2021-06-08 00:00:24.469000+00:00,NaN,NaN,NaN,46.0
2021-06-08 00:01:05.046000+00:00,NaN,NaN,NaN,45.0
2021-06-08 00:01:19.484000+00:00,NaN,NaN,NaN,46.0
2021-06-08 00:01:26.833000+00:00,NaN,NaN,NaN,45.0
...,...,...,...,...
2021-08-15 02:23:47.042000+00:00,371.0,420.0,0.0,32.0
2021-08-15 02:23:49.790000+00:00,371.0,420.0,1.0,32.0
2021-08-15 02:23:53.729000+00:00,371.0,420.0,0.0,32.0


In [40]:
df_norm = df_ffill.copy()
for col in df_ffill.columns:
    df_norm[col] = (df_ffill[col]-df_ffill[col].min())/(df_ffill[col].max()-df_ffill[col].min())
#     df_ffill[col+'_norm'] = (df_ffill[col]-df_ffill[col].min())/(df_ffill[col].max()-df_ffill[col].min())

In [45]:
df_norm.index.min()

Timestamp('2021-06-08 00:00:20.201000+0000', tz='UTC')

In [52]:
df_norm[~df_norm['sens_1'].isnull()]

tag_key,sens_1,sens_2,sens_4,sens_5
created_timestamp,,,,
2021-07-05 00:00:24.987000+00:00,0.004651,0.047619,NaN,0.004301
2021-07-05 00:00:27.833000+00:00,0.000000,0.045549,NaN,0.004301
2021-07-05 00:00:30.792000+00:00,0.000000,0.047619,NaN,0.004301
2021-07-05 00:00:45.338000+00:00,0.000000,0.045549,NaN,0.004301
2021-07-05 00:00:48.289000+00:00,0.000000,0.047619,NaN,0.004301
...,...,...,...,...
2021-08-15 02:23:47.042000+00:00,0.795349,0.797101,0.000000,0.004301
2021-08-15 02:23:49.790000+00:00,0.795349,0.797101,0.017544,0.004301
2021-08-15 02:23:53.729000+00:00,0.795349,0.797101,0.000000,0.004301


In [54]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px

app = Dash(__name__)

df = df_norm
sensor_info = {'Sensor 1': 'sens_1',
              'Sensor 2': 'sens_2',
              'Sensor 4': 'sens_4',
              'Sensor 5': 'sens_5',}

sensor_list = list(sensor_info.keys())

app.layout = html.Div([
    html.H1(children='Normalised Sensor Data Visualisation', style={'textAlign':'center'}),
    dcc.Dropdown(sensor_list, sensor_list[0], id='dropdown-selection'),
    dcc.Graph(id='graph-content')
])

@callback(
    Output('graph-content', 'figure'),
    Input('dropdown-selection', 'value')
)

def update_graph(sensor):
    sensor_tag = sensor_info[sensor]
    dff = df[[sensor_tag]]
#     dff = dff[~dff[sensor_tag].isnull()]
    
    fig = px.line(dff,
                  x=dff.index,
                  y=sensor_tag,
                  labels={'created_timestamp':'Created Timestamp',
                            sensor_tag: sensor})
    fig.update_xaxes(range=[dff.index.min(), dff.index.max()])

    return fig


if __name__ == '__main__':
    app.run(debug=True)